In [1]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [2]:
loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [3]:
logger_starter = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_starter = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_starter = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_starter = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)


picamera_starter = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

Making discoverable...
Starting Serial Port Profile...
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 89)


In [4]:

two_wheel_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

angular_speed_control_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)


bluetooth_control_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

picamera_starter.register_outgoing_rpc(
    dict(
        log=logger_starter.incoming_rpcs['log'],
        setup_video_saver=logger_starter.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_starter.incoming_rpcs['save_video_frame']
        )
)

two_wheel_starter.register_incoming_samples(
    angular_speed_control_starter.outgoing_sample_readers
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_sample_readers
)


bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

In [5]:
bluetooth_control_starter.start()
logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
picamera_starter.start()

/home/kawa/projects/car1/src/components/__init__.py:227: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")


/home/kawa/projects/car1/src/components/two_wheels.py:25: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:79: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:80: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
[3:02:23.936244554] [54324]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[3:02:23.983733355] [54395]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[3:02:23.987318752] [54395]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0

saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2



/home/kawa/projects/car1/src/components/__init__.py:227: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")


saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2saving: Picamera2V2

saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: Picamera2V2
saving: 

In [6]:
bt_ser.terminate()
bluetooth_control_starter.process.terminate()
logger_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
picamera_starter.process.terminate()

In [20]:
picamera_starter.process.terminate()

In [21]:

picamera_starter = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)


In [22]:
picamera_starter.register_outgoing_rpc(
    dict(
        log=logger_starter.incoming_rpcs['log'],
        setup_video_saver=logger_starter.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_starter.incoming_rpcs['save_video_frame']
        )
)

In [23]:
picamera_starter.start()

[2:59:13.169173774] [53338]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[2:59:13.204652300] [53352]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[2:59:13.207294058] [53352]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media4 and ISP device /dev/media1
[2:59:13.207363335] [53352]  INFO RPI pipeline_base.cpp:1104 Using configuration file '/usr/local/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[2:59:13.213801576] [53338]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[2:59:13.241835431] [53355]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[2:59:13.244569114] [53355]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media4 and ISP device /dev/media1
[2:59:13.244639909] [53355]  INFO RP

In [39]:
import plotly.express as px

px.imshow(picamera_starter.outgoing_sample_readers[0]()[..., ::-1]/255)

In [28]:
picamera_starter.outgoing_sample_readers[0]()/255

array([[[0.12941176, 0.16470588, 0.16862745],
        [0.1254902 , 0.18431373, 0.17647059],
        [0.11764706, 0.18431373, 0.2       ],
        ...,
        [0.41568627, 0.34117647, 0.28627451],
        [0.36470588, 0.2745098 , 0.21568627],
        [0.30980392, 0.25490196, 0.19215686]],

       [[0.38823529, 0.40392157, 0.38431373],
        [0.4       , 0.42352941, 0.38823529],
        [0.39215686, 0.42352941, 0.39607843],
        ...,
        [0.5254902 , 0.42745098, 0.37647059],
        [0.51764706, 0.42352941, 0.37254902],
        [0.51764706, 0.43529412, 0.3372549 ]],

       [[0.45098039, 0.47058824, 0.42745098],
        [0.47843137, 0.47058824, 0.41960784],
        [0.45098039, 0.4627451 , 0.42352941],
        ...,
        [0.56862745, 0.44313725, 0.36470588],
        [0.56470588, 0.45882353, 0.35294118],
        [0.56470588, 0.44313725, 0.33333333]],

       ...,

       [[0.70196078, 0.70980392, 0.63921569],
        [0.70588235, 0.71764706, 0.65882353],
        [0.69803922, 0

In [8]:
logger_starter.incoming_rpcs


{'log': <components.CallChannel at 0x7f6a88cfe0>,
 'setup_video_saver': <components.CallChannel at 0x7f6a88cd40>,
 'save_video_frame': <components.CallChannel at 0x7f6a88e690>,
 'get_logger': <components.CallChannel at 0x7f6a88e900>}

## Two Wheels

In [3]:
logger_starter = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_starter = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_starter = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_starter = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)


picamera_starter = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((114, 64), dtype=np.uint8)])
)


bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)

# sample
two_wheel_starter.register_incoming_samples(
    angular_speed_control_starter.outgoing_sample_readers
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_sample_readers
)

bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

# rpcs
logger_triplet = dict(
    log=logger_starter.incoming_rpcs['log'], 
    log_time=logger_starter.incoming_rpcs['log_time'], 
    increment_index=logger_starter.incoming_rpcs['increment_index'], 
)

picamera_starter.register_outgoing_rpc(
    logger_starter.incoming_rpcs
)


two_wheel_starter.register_outgoing_rpc(
    logger_triplet
)
angular_speed_control_starter.register_outgoing_rpc(
    logger_triplet
)
bluetooth_control_starter.register_outgoing_rpc(
    logger_triplet
)




logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
bluetooth_control_starter.start()
picamera_starter.start()



Making discoverable...
Starting Serial Port Profile...


/home/kawa/projects/car1/src/components/two_wheels.py:23: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:79: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:80: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/__init__.py:222: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/__init__.py:222: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")

saving: TwoWheelsV3saving: TwoWheelsV3

saving: BlueToothCarControlSPP_V2saving: BlueToothCarControlSPP_V2saving: BlueToothCarControlSPP_V2




In [17]:
angular_speed_control_starter.outgoing_sample_readers[0]()

100.0

In [6]:
angular_speed_control_starter.outgoing_sample_readers[0]()

0.0

In [7]:
angular_speed_control_starter.outgoing_sample_readers[1]()

0.0

In [ ]:
two_wheel_starter.

In [18]:
angular_speed_control_starter.get_outgoing_sample_writers()[0](100)

In [11]:
angular_speed_control_starter.get_outgoing_sample_writers()[1](0)

In [5]:
picamera_starter.get_outgoing_sample_writers()[0](np.zeros((114, 64), dtype=np.uint8))

In [6]:
picamera_starter.outgoing_sample_readers[0]()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [4]:



logger_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
bluetooth_control_starter.process.terminate()
picamera_starter.process.terminate()



In [13]:
LoggerComponent.handler_roles

{'rpc': [], 'sampler': [], 'sample_producer': [], 'loop': []}

In [8]:
logger_starter.incoming_rpcs

{}

In [4]:

logger_starter.register_incoming_events(
    dict(
        log_handler=[
            two_wheel_starter.outgoing_events['logging'], 
            angular_speed_control_starter.outgoing_events['logging'], 
            bluetooth_control_starter.outgoing_events['logging'], 
            picamera_starter.outgoing_events['logging'], 
        ]
    )
)



two_wheel_starter.register_incoming_samples(
    angular_speed_control_starter.outgoing_samples
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_samples
)

bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
bluetooth_control_starter.start()
picamera_starter.start()



/home/kawa/projects/car1/src/components/two_wheels.py:24: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:75: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:76: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/__init__.py:185: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/__init__.py:185: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")

saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Pi

In [5]:

logger_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
bluetooth_control_starter.process.terminate()
picamera_starter.process.terminate()



# ML Component


In [186]:
ml_out, ml_starter = default_component_process_starter_v2(
    ImageMLController, 
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)
ml = ml_starter(cam_out)

In [194]:
ml.terminate()

In [3]:
d = {'a':1, 'b':False}

if b:= d.get('b'):
    print (b)